In [1]:
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install llama-index-tools-tavily-research
!pip install llama-index-tools-finance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os

API_KEY = ""
os.environ["OPENAI_API_KEY"] = API_KEY

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:

papers = [
    "amazon_10k.pdf",
    "microsoft_10k.pdf",
]

In [5]:

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

In [7]:
from pathlib import Path
from utils import get_doc_tools

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: amazon_10k.pdf
Getting tools for paper: microsoft_10k.pdf


In [8]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec
tavily_tool = TavilyToolSpec(
    api_key="",
)

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.tools.finance import FinanceAgentToolSpec

POLYGON_API_KEY = ""
FINNHUB_API_KEY = ""
ALPHA_VANTAGE_API_KEY = ""
NEWSAPI_API_KEY = ""


fin_tool = FinanceAgentToolSpec(
        POLYGON_API_KEY,
        FINNHUB_API_KEY,
        ALPHA_VANTAGE_API_KEY,
        NEWSAPI_API_KEY,
    )

In [11]:
tavily_tool_list = tavily_tool.to_tool_list()
fin_tool_list = fin_tool.to_tool_list()

In [12]:
for tool in initial_tools:
    print(tool.metadata)

ToolMetadata(description='vector_tool_amazon_10k(query: str, page_numbers: Optional[List[str]] = None) -> str\nUse to answer questions over a given paper.\n    \n        Useful if you have specific questions over the paper.\n        Always leave page_numbers as None UNLESS there is a specific page you want to search for.\n    \n        Args:\n            query (str): the string query to be embedded.\n            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE \n                if we want to perform a vector search\n                over all pages. Otherwise, filter by the set of specified pages.\n        \n        ', name='vector_tool_amazon_10k', fn_schema=<class 'llama_index.core.tools.utils.vector_tool_amazon_10k'>, return_direct=False)
ToolMetadata(description='Useful for summarization questions related to amazon_10k', name='summary_tool_amazon_10k', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(desc

In [13]:
for tool in fin_tool_list:
    print(tool.metadata)

ToolMetadata(description="find_similar_companies(symbol: str) -> List[str]\nGiven a stock's ticker symbol, returns a list of similar companies.", name='find_similar_companies', fn_schema=<class 'llama_index.core.tools.utils.find_similar_companies'>, return_direct=False)
ToolMetadata(description="get_earnings_history(symbol: str) -> pandas.core.frame.DataFrame\nGiven a stock's ticker symbol, returns a dataframe storing actual and estimated earnings over past K quarterly reports.", name='get_earnings_history', fn_schema=<class 'llama_index.core.tools.utils.get_earnings_history'>, return_direct=False)
ToolMetadata(description="get_stocks_with_upcoming_earnings(num_days_from_now: int, only_sp500: bool) -> pandas.core.frame.DataFrame\nReturns a pandas dataframe containing all stocks which are announcing earnings in upcoming days.\n\n        Arguments:\n         num_days_from_now: only returns stocks which announcing earnings from today's date to num_days_from_now.\n         only_sp500: only

In [14]:
initial_tools += (tavily_tool_list + fin_tool_list)

In [15]:
print(initial_tools)

[<llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c9a131d20>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x7d9c977d0c40>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c967fab30>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x7d9c96b30ac0>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c9548c700>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c9b885840>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c9b887fd0>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c95e2b2e0>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c95e2b910>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c95e29c30>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c95e29c00>, <llama_index.core.tools.function_tool.FunctionTool object at 0x7d9c95e2b280>, <llama_index.core.tools.function_tool.FunctionTool object a

In [16]:
len(initial_tools)

20

In [17]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [18]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [26]:
response = agent.query(
    "Tell me all the details about microsoft with the perspective of investor ",
)

Added user message to memory: Tell me all the details about microsoft with the perspective of investor 
=== Calling Function ===
Calling function: vector_tool_microsoft_10k with args: {"query": "investor perspective"}
=== Function Output ===
Investors would likely be concerned about the risks associated with acquisitions and other transactions, as well as the competitive landscape in the markets for software, devices, and cloud-based services. They may also pay attention to the impact of economic conditions and geopolitical factors on the company's operations and financial performance. Additionally, investors may be interested in how the company is managing its costs, investments in strategic areas, and workforce restructuring to align with revenue and customer demand.
=== LLM Response ===
Investors in Microsoft may be concerned about risks related to acquisitions, competitive landscape in software, devices, and cloud-based services markets, economic conditions, and geopolitical factor

In [27]:
print(response)

Investors in Microsoft may be concerned about risks related to acquisitions, competitive landscape in software, devices, and cloud-based services markets, economic conditions, and geopolitical factors. They may also focus on cost management, strategic investments, and workforce restructuring to align with revenue and customer demand.
